# Parse LabelMe Annotations and Images

# 1. Import Essential Libraries

In [1]:
import numpy as np
import os
from PIL import Image, ImageDraw
import cv2
import xmltodict
import sys

# 2. Single Image Parsing

In [2]:
def xml2dict(xml_path, product):
        print xml_path
        file_exist = os.path.isfile(xml_path)    # True
        # print xml_path
        pts = []
        xpts = []
        ypts = []

        if file_exist:

                with open(xml_path) as fd:
                        label_dict = xmltodict.parse(fd.read())
        else:
                return file_exist, xpts, ypts



        try:
                for object_ in label_dict['annotation']['object']:
                        if object_['name'] == product  and object_['deleted'] == '0':
                                pts.append(object_['polygon']['pt'])

                                for pts_idx in range(0,len(object_['polygon']['pt'])):
                                        xpts.append(int(object_['polygon']['pt'][pts_idx]['x']))
                                        ypts.append(int(object_['polygon']['pt'][pts_idx]['y']))

                                return file_exist, xpts,ypts

        except:

                try: ## xml exist and have label
                        object_ = label_dict['annotation']['object']
                        #print object_
                        if object_['name'] == product  and object_['deleted'] == '0':

                                #print object_['polygon']['pt']                 
                                pts.append(object_['polygon']['pt'])
                                # print pts
                                for pts_idx in range(0,len(object_['polygon']['pt'])):
                                        xpts.append(int(object_['polygon']['pt'][pts_idx]['x']))
                                        ypts.append(int(object_['polygon']['pt'][pts_idx]['y']))
                                print "2"
                                return file_exist, xpts,ypts


                        else: #xml exist, labeld has been deleted
                                print "xml exist, labeld has been deleted"
                                xpts = []
                                ypts = []
                                file_exist = False
                                return file_exist, xpts, ypts




                except: ## xml exist, but no label polygon
                        xpts = []
                        ypts = []
                        file_exist = False
                        print "no xml"
                        return file_exist, xpts, ypts


        else:
                file_exist = False
                return file_exist, xpts, ypts

# This function help you to prepare polygon format needed by PIL.
def merge_x_y_pts(file_exist, xpts, ypts):
        mixed_x_y = []
        if file_exist:
                for pt_idx in range(0,len(xpts)):
                        mixed_x_y.append(xpts[pt_idx])
                        mixed_x_y.append(ypts[pt_idx])

        return mixed_x_y



# This function help you to return a mask, In this mask, only the pixels in bounding box area are set to 255, other pixels are set to 0.
def poly2mask(mixed_x_y, fill_value=255):
        width = 640
        height = 480
        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(mixed_x_y, outline=1, fill=fill_value)
        mask = np.array(img)
        return mask


# This function help you to crop mask area from original iamge.
def mask_image_crop(mask, img):
        H = 480
        W = 640
        min_x = 0
        min_y = 0
        max_x = 0
        max_y = 0

        isarea = False
        area = np.nonzero(mask)

        if len(area[0]) is 0 and len(area[1]) is 0:

                return isarea, area

        else:
                isarea = True
                min_x = min(area[1])
                min_y = min(area[0])
                max_x = max(area[1])
                max_y = max(area[0])
                print min_x,min_y
                print max_x,max_y

                crop_h = max_y - min_y
                crop_w = max_x - min_x

                crop_img = img[min_y:min_y + crop_h, min_x:min_x + crop_w]
                return isarea, crop_img




# 3. Setup Path of Annotations, Images, Output, and Classes

* modified three folder pathes and clsses depending on your case

In [25]:
annotations_path = '/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90'
images_path = '/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Images/users/argrobotx/robotx_duckietown/wama_minus_90'
output_cropped_images_path= '/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images'
class_name = ['blue_circle','green_triangle','red_cross']

# 4. Create Folders of Cropped Images

In [26]:
cropped_image_class_path = []
for i in range(len(class_name)):
    directory_path = output_cropped_images_path + '/' + class_name[i] + '/'
    directory = os.path.dirname(directory_path)
    try:
        os.stat(directory)
        print directory, ' exists'
    except:
        os.mkdir(directory)
        print 'creat ', directory
    cropped_image_class_path.append(directory_path)


creat  /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/blue_circle
creat  /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/green_triangle
creat  /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/red_cross


# 5. Start Parsing Images and xml files

In [27]:
for image_file in os.listdir(images_path):
    find_xml = False
    img_path = images_path + '/' + image_file
    image_file_split = image_file.split('.')
    xml_file = image_file_split[0] + '.xml'
    print img_path

    # find relevant annotation file
    for annotation_file in os.listdir(annotations_path):
        if xml_file == annotation_file:
            find_xml = True
            xml_path = annotations_path + '/' + annotation_file
    
    if find_xml == True:
        img = cv2.imread(img_path)
        for i in range(len(class_name)):
            
            cropped_path = cropped_image_class_path[i] + '/' + image_file_split[0] + '_' + class_name[i] + '.jpg'

            file_exist, xpts, ypts = xml2dict(xml_path, class_name[i])
            if file_exist:
                print xpts, ypts
                mixed_x_y = merge_x_y_pts(file_exist, xpts, ypts)
                mask = poly2mask(mixed_x_y, fill_value=255)
                isarea, cropped = mask_image_crop(mask, img)

                if isarea is True:
                        print "write to %s" %(cropped_path)
                        cv2.imwrite(cropped_path, cropped)            
    

/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Images/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0000.jpg
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0000.xml
[426, 434, 434, 426] [136, 136, 123, 123]
426 123
434 136
write to /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/blue_circle//wama_-90_0000_blue_circle.jpg
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0000.xml
[450, 458, 458, 450] [142, 142, 129, 129]
450 129
458 142
write to /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/green_triangle//wama_-90_0000_green_triangle.jpg
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_

588 149
612 184
write to /home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Cropped_Images/blue_circle//wama_-90_0025_blue_circle.jpg
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0025.xml
xml exist, labeld has been deleted
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0025.xml
xml exist, labeld has been deleted
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Images/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0018.jpg
/home/arg_ws2/robotx_brianchuang_fg_training/data/placard/robotx_duckietown_wama_minus_90/Annotations/users/argrobotx/robotx_duckietown/wama_minus_90/wama_-90_0018.xml
[469, 479, 479, 469] [124, 124, 143, 143]
469 124
479 143
write to /home/arg_ws2/robotx_br